# 1. Investigating dask histogram pacakge 
https://examples.dask.org/applications/embarrassingly-parallel.html

In [ ]:
import dask_histogram as dh
import dask_histogram.boost as dhb
import dask.array as da
import time
import dask
from dask.distributed import Client, progress
print("Libraries were imported")

## 2. Create a distributed client

In [ ]:
client = Client(threads_per_worker=4, n_workers=10)
client

## 3. Create a costly simulation method to simulate a long process for dask to parallelize

In [ ]:
def costly_simulation():
    x = da.random.standard_normal(size=(100_000, 2), chunks=(10_000, 2))
    t = dhb.Histogram(
        dh.axis.Regular(10, -3, 3),
        dh.axis.Regular(10, -3, 3),
        storage=dh.storage.Double(),
    )
    t.fill(x)
    t.compute()

In [ ]:
start = time.time()
costly_simulation()
finish = time.time()

print(finish - start)

## 4. Create a list of 360 delayed calls to compute histograms

In [ ]:
def create_delayed_objects():
    results = []
    for i in range(0, 360):
        result = dask.delayed(costly_simulation)()
        results.append(result)
    return results

## 5. Call and time the execution of 360 histograms

In [ ]:
start = time.time()
dask.compute(*create_delayed_objects())
finish = time.time()

print("The 360 histograms took: ", finish - start)